In [1]:
transformer_state = {
    "soc": 0.42,          # Estimated SoC
    "soh": 0.93,          # Estimated SoH
    "temp": 305.0,        # Current temperature (K)
    "confidence": 0.91    # Optional
}

In [2]:
import numpy as np

In [3]:
# Battery and Safety Parameters

BATTERY_PARAMS = {
    "capacity_Ah": 50.0,
    "R0_nominal": 0.015,

    "V_max": 4.2,
    "V_min": 3.0,

    "T_amb": 298.0,
    "T_max": 333.0,

    "R_th": 1.5,
    "C_th": 500.0,

    "dt": 1.0
}

In [4]:
# OCV Model

def ocv_function(soc):
    return 3.0 + 1.2 * soc

In [5]:
# ECM step (SoH-aware resistance)
# Internal resistance increases as SoH drops

def ecm_step(soc, soh, current, dt, params):
    Q = params["capacity_Ah"] * 3600
    soc_next = soc + (current * dt) / Q
    soc_next = np.clip(soc_next, 0.0, 1.0)

    R0 = params["R0_nominal"] * (1.0 / soh)
    ocv = ocv_function(soc)

    voltage = ocv + current * R0
    return soc_next, voltage, R0

In [6]:
# Thermal Model

def thermal_step(temp, current, R0, params, dt):
    heat_gen = (current ** 2) * R0
    heat_loss = (temp - params["T_amb"]) / params["R_th"]

    temp_next = temp + (dt / params["C_th"]) * (heat_gen - heat_loss)
    return temp_next

In [7]:
# Degradation Proxy
# SoH loss accelerates at high temperature and current

def degradation_step(soh, current, temp, dt):
    stress = abs(current) * max(0.0, temp - 298.0)
    delta = 1e-8 * stress * dt
    soh_next = soh - delta
    return max(0.0, soh_next)

In [8]:
# Core simulation function (conditioned on Transformer output)
# This is the synthetic data generator used by the optimizer

def simulate_charging(current_profile, transformer_state, params, soc_target=0.9):
    soc = transformer_state["soc"]
    soh = transformer_state["soh"]
    temp = transformer_state["temp"]

    dt = params["dt"]
    peak_temp = temp

    for t, current in enumerate(current_profile):
        soc, voltage, R0 = ecm_step(soc, soh, current, dt, params)
        temp = thermal_step(temp, current, R0, params, dt)
        soh = degradation_step(soh, current, temp, dt)

        if voltage > params["V_max"] or voltage < params["V_min"]:
            return "voltage"

        if temp > params["T_max"]:
            return "temp"

        if soc >= soc_target:
            break

    if soc < soc_target:
        return "soc"

    charging_time = t * dt
    soh_loss = transformer_state["soh"] - soh
    return charging_time, peak_temp, soh_loss


In [9]:
# Fitness function (single-objective GA)

def fitness_function(sim_result):
    if sim_result is None:
        return -1e6

    charging_time, peak_temp, soh_loss = sim_result
    temp_penalty = max(0.0, peak_temp - 318.0)

    return (
        -charging_time
        - 1e4 * soh_loss
        - 10.0 * temp_penalty
    )

In [10]:
# Genetic Algorithm (GA)
# Hyperparameters

POP_SIZE = 40
N_GENERATIONS = 25
HORIZON_SEC = 300          # 5 minutes
DT = 1.0                  # seconds
N_GENES = int(HORIZON_SEC / DT)

I_MIN, I_MAX = 5.0, 80.0  # charging current bounds (A)

ELITE_FRAC = 0.2
MUTATION_PROB = 0.1
MUTATION_STD = 5.0

In [11]:
# GA helpers

def initialize_population():
    return [
        np.random.uniform(I_MIN, I_MAX, N_GENES)
        for _ in range(POP_SIZE)
    ]


def crossover(p1, p2):
    point = np.random.randint(1, N_GENES - 1)
    return np.concatenate([p1[:point], p2[point:]])


def mutate(individual):
    for i in range(N_GENES):
        if np.random.rand() < MUTATION_PROB:
            individual[i] += np.random.normal(0, MUTATION_STD)
    return np.clip(individual, I_MIN, I_MAX)

In [12]:
# GA loop
population = initialize_population()
best_fitness_history = []

for gen in range(N_GENERATIONS):

    fitnesses = []
    for individual in population:
        sim_result = simulate_charging(
            individual,
            transformer_state,
            BATTERY_PARAMS
        )
        if isinstance(sim_result, str):
            print("Failure:", sim_result)
            #fitnesses.append(fitness_function(sim_result))

    fitnesses = np.array(fitnesses)

    # Track best
    best_idx = np.argmax(fitnesses)
    best_fitness_history.append(fitnesses[best_idx])

    # Selection (elitism)
    elite_count = int(ELITE_FRAC * POP_SIZE)
    elite_indices = np.argsort(fitnesses)[-elite_count:]
    elites = [population[i] for i in elite_indices]

    # Create next generation
    next_population = elites.copy()
    while len(next_population) < POP_SIZE:
        idx1, idx2 = np.random.choice(len(elites), 2, replace=False)
        p1, p2 = elites[idx1], elites[idx2]
        child = crossover(p1, p2)
        child = mutate(child)
        next_population.append(child)

    population = next_population

    print(f"Generation {gen+1}/{N_GENERATIONS} | Best fitness: {best_fitness_history[-1]:.4f}")

Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage
Failure: voltage


ValueError: attempt to get argmax of an empty sequence

In [ ]:
# Final best solution
best_charging_profile = population[np.argmax(fitnesses)]

print("GA optimization complete.")

GA optimization complete.


In [ ]:
# Debug: constant low current
test_profile = np.ones(300) * 10.0  # 10A gentle charge

result = simulate_charging(
    test_profile,
    transformer_state,
    BATTERY_PARAMS
)

print(result)

(300.0, 305.0, 0.00018575295202061248)
